In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "Data/Updated_Scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")] 

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score 

In [7]:
def read_stats (soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df
    

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container") [0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0] 
    return season

In [9]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1, :]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format ="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 1 == 0:
        print(f"{len(games)} / {len(box_scores)}")
    
    



1 / 11
2 / 11
3 / 11
4 / 11
5 / 11
6 / 11
7 / 11
8 / 11
9 / 11
10 / 11
11 / 11


In [10]:
games_df = pd.concat(games, ignore_index=True)

In [11]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,47.0,90.0,0.522,15.0,35.0,0.429,4.0,8.0,...,100.0,28.1,171.0,123.0,PHO,123,1,2024,2024-02-25,False
1,240.0,240.0,44.0,93.0,0.473,17.0,40.0,0.425,18.0,19.0,...,15.0,30.0,232.0,136.0,LAL,113,0,2024,2024-02-25,True
2,240.0,240.0,35.0,84.0,0.417,14.0,38.0,0.368,8.0,18.0,...,20.0,27.7,158.0,105.0,ATL,109,1,2024,2024-02-25,False
3,240.0,240.0,41.0,90.0,0.456,12.0,29.0,0.414,15.0,17.0,...,50.0,29.1,186.0,122.0,ORL,92,0,2024,2024-02-25,True
4,240.0,240.0,41.0,95.0,0.432,13.0,40.0,0.325,19.0,21.0,...,58.1,31.3,158.0,123.0,NOP,106,1,2024,2024-02-25,True
5,240.0,240.0,40.0,88.0,0.455,16.0,41.0,0.390,10.0,14.0,...,20.0,27.6,200.0,117.0,CHI,114,0,2024,2024-02-25,False
6,240.0,240.0,43.0,79.0,0.544,18.0,37.0,0.486,15.0,18.0,...,33.3,29.9,211.0,134.0,PHI,98,1,2024,2024-02-25,True
7,240.0,240.0,36.0,97.0,0.371,16.0,46.0,0.348,10.0,11.0,...,23.0,38.7,300.0,116.0,MIL,119,0,2024,2024-02-25,False
8,240.0,240.0,42.0,91.0,0.462,13.0,31.0,0.419,12.0,19.0,...,100.0,25.3,300.0,113.0,UTA,128,1,2024,2024-02-25,False
9,240.0,240.0,46.0,90.0,0.511,17.0,35.0,0.486,19.0,22.0,...,66.7,40.5,162.0,127.0,SAS,109,0,2024,2024-02-25,True


In [12]:
games_df.to_csv('updated_nba_boxscores.csv')